In [1]:
import numpy as np
import os
import _pickle as pkl
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.pairwise import chi2_kernel
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
FEATURE_DIR = "features"
FUSION_DIR = "features/double_fusion"
with open("list/all.video") as f:
    all_videos = [l.strip() for l in f.readlines()]

with open('list/train.video') as f:
    train_videos = [l.strip() for l in f.readlines()]
    
with open('list/val.video') as f:
    val_videos = [l.strip() for l in f.readlines()]

with open('../all_test.video') as f:
    test_videos = [l.strip() for l in f.readlines()]
    
# train labels
train_labels = {}
with open("../all_trn.lst") as f:
    for line in f.readlines():
        file, label = line.strip().split()
        train_labels[file] = label
        
# val labels
val_labels = {}
with open("../all_val.lst") as f:
    for line in f.readlines():
        file, label = line.strip().split()
        val_labels[file] = label

In [3]:
def load_features(video_list,feature_type):
    features = [] 
    for file in tqdm(video_list):
        if feature_type == 'soundnet':
            feature_file = os.path.join(FEATURE_DIR,"soundnet",file+".feats")
            with open(feature_file,"rb") as f:
                features.append(np.loadtxt(f,delimiter=';'))
        elif feature_type == 'resnet50':
            feature_file = os.path.join(FEATURE_DIR,"resnet50",file+".npy")   
            with open(feature_file,"rb") as f:
                features.append(np.load(f))
        else:
            feature_file = os.path.join(FEATURE_DIR,"places",file+".npy")   
            with open(feature_file,"rb") as f:
                features.append(np.load(f))
                
    features = np.vstack(features)
    return features

def load_labels(video_list,label_map):
    labels = []
    for video in tqdm(video_list):
        labels.append(label_map[video])
        
    return np.array(labels)

train_soundnet_features = load_features(train_videos,'soundnet')
train_resnet_features = load_features(train_videos,'resnet50')
train_places_features = load_features(train_videos,'places')

train_y = load_labels(train_videos,train_labels)

val_soundnet_features = load_features(val_videos,'soundnet')
val_resnet_features = load_features(val_videos,'resnet50')
val_places_features = load_features(val_videos,'places')

val_y = load_labels(val_videos,val_labels)

test_soundnet_features = load_features(test_videos,'soundnet')
test_resnet_features = load_features(test_videos,'resnet50')
test_places_features = load_features(test_videos,'places')

100%|██████████| 1699/1699 [00:02<00:00, 793.51it/s]


In [4]:
def generate_pairwise_features(soundnet, resnet, places):
    sound_res = np.hstack((soundnet,resnet))
    sound_places = np.hstack((soundnet,places))
    res_places = np.hstack((resnet,places))
    all_comb = np.hstack((soundnet,resnet,places))
    
#     return [sound_res, sound_places, res_places, all_comb]
#     return [sound_res, res_places]
    return sound_res,res_places

train_SR_features,train_RP_features = generate_pairwise_features(train_soundnet_features,train_resnet_features,train_places_features)
val_SR_features,val_RP_features = generate_pairwise_features(val_soundnet_features,val_resnet_features,val_places_features)
test_SR_features, test_RP_features = generate_pairwise_features(test_soundnet_features,test_resnet_features,test_places_features)

In [46]:
def downsample_data(train_x,train_y,val_x,val_y,combine):
#     print("Downsampling")
# Combine train and validation set into a single train dataset
    if combine:
        all_train_x = np.vstack([train_x,val_x])
        all_train_y = np.concatenate((train_y,val_y))

        # Shuffle dataset
        idxs = [i for i in range(all_train_x.shape[0])]
        np.random.shuffle(idxs)

        all_train_x = all_train_x[idxs]
        all_train_y = all_train_y[idxs]
    else:
        all_train_x = train_x
        all_train_y = train_y

    positive_examples = all_train_x[all_train_y!='NULL']
    negative_examples = all_train_x[all_train_y=='NULL']

    positive_labels = all_train_y[all_train_y!='NULL']
    negative_labels = all_train_y[all_train_y == 'NULL']

    np.random.shuffle(negative_examples)
    negative_examples = negative_examples[:len(positive_labels)//2]
    negative_labels = negative_labels[:len(positive_labels)//2]

    combined_data = np.vstack((positive_examples,negative_examples))
    combined_labels = np.concatenate((positive_labels,negative_labels))
    
    return combined_data,combined_labels

COMBINE = False
soundnet_x,soundnet_y = downsample_data(train_soundnet_features,train_y,val_soundnet_features,val_y,combine=COMBINE)
resnet_x, resnet_y = downsample_data(train_resnet_features,train_y,val_resnet_features,val_y,combine=COMBINE)
places_x, places_y = downsample_data(train_places_features,train_y,val_places_features,val_y,combine=COMBINE)
sr_x, sr_y = downsample_data(train_SR_features,train_y,val_SR_features,val_y,combine=COMBINE)
rp_x, rp_y = downsample_data(train_RP_features,train_y,val_RP_features,val_y,combine=COMBINE)

idxs = [i for i in range(len(soundnet_x))]
np.random.shuffle(idxs)

soundnet_x = soundnet_x[idxs]
soundnet_y = soundnet_y[idxs]

resnet_x = resnet_x[idxs]
resnet_y = resnet_y[idxs]

places_x = places_x[idxs]
places_y = places_y[idxs]

sr_x = sr_x[idxs]
sr_y = sr_y[idxs]

rp_x = rp_x[idxs]
rp_y = rp_y[idxs]

print("Downsampling complete")

Downsampling complete


In [47]:
MODEL_USED = 'mlp'
MODEL_DIR = "models"
MODEL_NAME = 'df_exp1'
EVENTS = ["NULL","P001","P002","P003"]
EVENTS_MAP = {"NULL":0,"P001":1,"P002":2,"P003":3}

def train_classifier(combined_data,combined_labels):
# Train 3 separate models for each event
# MLP parameters


    hidden_layers = (1024,2048)
    lr = 1e-3
    batch_size = 50
    alpha = 1e-5
    loss = []


    # Random Forest params
    num_estimators = 50
    max_depth = 10
    event_y = np.array([EVENTS_MAP[k] for k in combined_labels])
    
    if MODEL_USED == 'rf':
        model = RandomForestClassifier(n_estimators=num_estimators,max_depth=max_depth)
#         model = GradientBoostingClassifier(n_estimators=num_estimators,max_depth=max_depth)
    elif MODEL_USED == 'mlp':
        model = MLPClassifier(hidden_layer_sizes=hidden_layers,
                         alpha=alpha,
                         batch_size=batch_size,
                         learning_rate_init=lr,
                         max_iter=2000,
                            verbose=False,
                         )
    model.fit(combined_data,event_y)
    if MODEL_USED== 'mlp':
        loss.append(model.loss_)
        print("Loss : {} ".format(model.loss_))
        
    return model

soundnet_classifier = train_classifier(soundnet_x,soundnet_y)
resnet_classifier = train_classifier(resnet_x,resnet_y)
places_classifier = train_classifier(places_x,places_y)
sr_classifier = train_classifier(sr_x,sr_y)
rp_classifier = train_classifier(rp_x,rp_y)

Loss : 0.02292690350559003 
Loss : 0.0004771475513751168 
Loss : 0.000685666772717504 
Loss : 0.0005841141290220448 
Loss : 0.0007679877114349029 


In [48]:
soundnet_post_features = soundnet_classifier.predict_log_proba(soundnet_x)
resnet_post_features = resnet_classifier.predict_log_proba(resnet_x)
places_post_features = places_classifier.predict_log_proba(places_x)
sr_post_features = sr_classifier.predict_log_proba(sr_x)
rp_post_features = rp_classifier.predict_log_proba(rp_x)

In [49]:
post_features = np.hstack((soundnet_post_features,resnet_post_features,
                    places_post_features,sr_post_features,rp_post_features))
post_labels = soundnet_y

In [50]:
# Train 3 separate models for each event
# MLP parameters
MODEL_USED = 'mlp'
MODEL_DIR = "models"
MODEL_NAME = 'lf_exp1'
EVENTS = ["NULL","P001","P002","P003"]

hidden_layers = (512)
lr = 1e-3
batch_size = 100
alpha = 1e-5
loss = []


# Random Forest params
num_estimators = 3000
max_depth = 30

for event in tqdm(EVENTS[:]):
    # Train for event
    event_y = (post_labels==event).astype('int')
    if MODEL_USED == 'rf':
        model = RandomForestClassifier(n_estimators=num_estimators,max_depth=max_depth)
#         model = GradientBoostingClassifier(n_estimators=num_estimators,max_depth=max_depth)
    elif MODEL_USED == 'mlp':
        model = MLPClassifier(hidden_layer_sizes=hidden_layers,
                         alpha=alpha,
                         batch_size=batch_size,
                         learning_rate_init=lr,
                         max_iter=2000,
                            verbose=False,
                         )
    model.fit(post_features,event_y)
    if MODEL_USED== 'mlp':
        loss.append(model.loss_)
        print("Event {} loss : {} ".format(event,model.loss_))
    # Save model
    model_path = os.path.join(MODEL_DIR,MODEL_NAME+"_" +event+".model")
    with open(model_path,"wb") as o:
        pkl.dump(model,o)

 50%|█████     | 2/4 [00:00<00:00, 11.32it/s]

Event NULL loss : 0.00033421912005690086 
Event P001 loss : 9.780088712169413e-05 
Event P002 loss : 5.663116167557355e-06 


100%|██████████| 4/4 [00:00<00:00, 12.74it/s]

Event P003 loss : 0.0034238954349864637 


In [51]:
PRED_DIR = 'pred/DF'
MODE = 'DF'

for event in EVENTS[:]:
    # Load model for event and predict
    model_path = os.path.join(MODEL_DIR,MODEL_NAME+"_" +event+".model")
    with open(model_path,"rb") as l:
        model = pkl.load(l,encoding='bytes')
        val_pred = os.path.join(PRED_DIR,MODE+"_val_"+event+".csv")
        test_pred = os.path.join(PRED_DIR,MODE+"_"+event+".csv")
        
        with open(test_pred,"w") as o: 
            test_soundnet_post_features = soundnet_classifier.predict_log_proba(test_soundnet_features)
            test_resnet_post_features = resnet_classifier.predict_log_proba(test_resnet_features)
            test_places_post_features = places_classifier.predict_log_proba(test_places_features)
            test_sr_post_features = sr_classifier.predict_log_proba(test_SR_features)
            test_rp_post_features = rp_classifier.predict_log_proba(test_RP_features)
            
            test_post_features = np.hstack((test_soundnet_post_features,test_resnet_post_features,
                                            test_places_post_features,test_sr_post_features,test_rp_post_features))
            
            prediction = model.predict_proba(test_post_features)
            for p in tqdm(prediction):
#                 print(p)
                o.write("{}\n".format(p[1]))
#           prediction = model.decision_function(feat)
#           o.write("{}\n".format(prediction[0]))


                    
        with open(val_pred,"w") as o: 
            val_soundnet_post_features = soundnet_classifier.predict_log_proba(val_soundnet_features)
            val_resnet_post_features = resnet_classifier.predict_log_proba(val_resnet_features)
            val_places_post_features = places_classifier.predict_log_proba(val_places_features)
            val_sr_post_features = sr_classifier.predict_log_proba(val_SR_features)
            val_rp_post_features = rp_classifier.predict_log_proba(val_RP_features)
            
            val_post_features = np.hstack((val_soundnet_post_features,val_resnet_post_features,
                                           val_places_post_features,val_sr_post_features,val_rp_post_features))
            
            prediction = model.predict_proba(val_post_features)
            for p in tqdm(prediction):
                o.write("{}\n".format(p[1]))
#               prediction = model.decision_function(feat)
#               o.write("{}\n".format(prediction[0]))
                     
p0 = open(PRED_DIR + "/" + MODE + "_val_NULL.csv").readlines()                    
p1 = open(PRED_DIR + "/" + MODE + "_val_P001.csv").readlines()
p2 = open(PRED_DIR + "/" + MODE + "_val_P002.csv").readlines()
p3 = open(PRED_DIR + "/" + MODE + "_val_P003.csv").readlines()

p0 = [float(p.strip()) for p in p0]
p1 = [float(p.strip()) for p in p1]
p2 = [float(p.strip()) for p in p2]
p3 = [float(p.strip()) for p in p3]

acc = [0,0,0,0]
total = [0,0,0,0]

THRESHOLD = 0
correct = 0
with open("../all_val.lst") as f:
    lines = f.readlines()
    for idx,line in enumerate(tqdm(lines)):
        filename, truth = line.strip().split()
        label = [p0[idx],p1[idx],p2[idx],p3[idx]]
        pred = np.argmax(label)
        
        if EVENTS[pred] == truth:
            acc[pred] += 1
        total[EVENTS.index(truth)] += 1
            
acc = np.array(acc)/np.array(total)
print("Event NULL val accuracy : {0:1.2f}\nEvent P001 val accuracy : {0:1.2f}\nEvent P002 val accuracy : {1:1.2f}\nEvent P003 val accuracy : {2:1.2f}\n"
      .format(acc[0],acc[1],acc[2],acc[3]))

# Write params in log
with open("logs.txt","a") as log:
    if MODEL_USED == 'mlp':
        log.write("{0}#{1}#{2}#{3}#{4}#{5:0.2f}#{6:0.2f}#{7:0.2f}#{8:0.2f}#{9:0.2f}#{10:0.2f}\n".format(MODEL_USED,hidden_layers,lr,batch_size,
                                                       alpha,loss[0],loss[1],loss[2],acc[0],acc[1],acc[2]))
    elif MODEL_USED == 'rf':
        log.write("{0}#{1}#{2}#{3:0.2f}#{4:0.2f}#{5:0.2f}\n".format(MODEL_USED,num_estimators,max_depth,acc[0],acc[1],acc[2]))


100%|██████████| 400/400 [00:00<00:00, 122910.01it/s]

Event NULL val accuracy : 0.65
Event P001 val accuracy : 0.65
Event P002 val accuracy : 0.87
Event P003 val accuracy : 0.89



In [52]:
!python eval.py list/P001_val_label pred/DF/DF_val_P001.csv
!python eval.py list/P002_val_label pred/DF/DF_val_P002.csv
!python eval.py list/P003_val_label pred/DF/DF_val_P003.csv

Evaluating the average precision (AP)
Average precision:  0.7989637360007185
Evaluating the average precision (AP)
Average precision:  0.9682407407407407
Evaluating the average precision (AP)
Average precision:  0.6738261550999769


In [53]:
p0 = open(PRED_DIR + "/" + MODE + "_NULL.csv").readlines()
p1 = open(PRED_DIR + "/" + MODE + "_P001.csv").readlines()
p2 = open(PRED_DIR + "/" + MODE + "_P002.csv").readlines()
p3 = open(PRED_DIR + "/" + MODE + "_P003.csv").readlines()

p0 = [float(p.strip())-0.3 for p in p0]
p1 = [float(p.strip()) for p in p1]
p2 = [float(p.strip()) for p in p2]
p3 = [float(p.strip()) for p in p3]

THRESHOLD = 0
with open((MODE+"_kaggle_prediction.csv"),"w") as o:
    o.write("VideoID,Label\n")
    for idx,video in enumerate(tqdm(test_videos)):
        label = [p0[idx],p1[idx],p2[idx],p3[idx]]
        idx = np.argmax(label)
        pred = idx
        o.write("{},{}\n".format(video,pred))

100%|██████████| 1699/1699 [00:00<00:00, 113143.59it/s]
